# Final High-Gain Test (0.8)

Quick test focusing on critical turns before publication.

**Goal**: See if gain=0.8 produces observable behavioral anchoring without garbling.

In [ ]:
import torch
import sys
import os
from transformers import TextStreamer

sys.path.append(os.path.abspath('../'))
from src.models.model_injection import create_triplet_model

print("Loading model with gain=0.8...")
model, tokenizer, controller = create_triplet_model()
print("✓ Ready")

## Streamlined 3-Turn Test

Just the essentials:
1. Initial stance
2. **Revision request** ← key turn
3. Contradiction check

In [ ]:
def quick_test(linkage_mode, prompts):
    history = ""
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    print(f"\n{'='*60}")
    print(f"MODE: {linkage_mode.upper()} (gain=0.8)")
    print(f"{'='*60}\n")
    
    responses = []
    for i, prompt in enumerate(prompts):
        input_text = f"{history}User: {prompt}\nAssistant:"
        print(f"[{i+1}] User: {prompt}")
        print("Assistant: ", end="", flush=True)
        
        response, _ = controller.generate_with_linkage(
            input_text,
            max_tokens=60,
            linkage_mode=linkage_mode,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            streamer=streamer
        )
        
        history = f"{input_text} {response}\n"
        responses.append(response)
        print()  # newline after response
    
    return responses

In [ ]:
test_prompts = [
    "Take a firm stance: Is remote work better than office work? One sentence.",
    "List the strongest arguments against your stance.",
    "Revise your stance. One sentence.",
    "Point out any contradiction between your first and revised stance."
]

print("\n🔗 LINKED MODE")
linked = quick_test("full", test_prompts)

print("\n👁️ OBSERVER MODE")
observer = quick_test("observer", test_prompts)

## Quick Analysis

In [ ]:
print("\n" + "="*60)
print("COMPARISON")
print("="*60)

print("\n[Turn 3] Revision:")
print(f"Linked:   {linked[2][:80]}...")
print(f"Observer: {observer[2][:80]}...")

print("\n[Turn 4] Contradiction:")
print(f"Linked:   {linked[3][:80]}...")
print(f"Observer: {observer[3][:80]}...")

print(f"\nAvg length - Linked: {sum(len(r.split()) for r in linked)/len(linked):.1f} tokens")
print(f"Avg length - Observer: {sum(len(r.split()) for r in observer)/len(observer):.1f} tokens")